In [ ]:
!pip install git+https://github.com/sjsu2024-data298-team6/ultralytics.git
!pip install boto3 tensorflowjs psycopg2-binary

import boto3
import botocore
from botocore.exceptions import ClientError
import os
from google.colab import userdata

os.environ['AWS_ACCESS_KEY_ID'] = userdata.get("AWS_ACCESS_KEY_ID")
os.environ['AWS_SECRET_ACCESS_KEY'] = userdata.get("AWS_SECRET_ACCESS_KEY")
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

  Cloning https://github.com/sjsu2024-data298-team6/ultralytics.git to /tmp/pip-req-build-9imh0lzd
  Running command git clone --filter=blob:none --quiet https://github.com/sjsu2024-data298-team6/ultralytics.git /tmp/pip-req-build-9imh0lzd
  Resolved https://github.com/sjsu2024-data298-team6/ultralytics.git to commit 9d0c4cadcce475aa5e143373a357a8da00729367
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
s3 = boto3.resource('s3')
BUCKET = "sjsu-fall24-data298-team6-bucket"
KEY = 'runs/yolo.zip'
bucket = s3.Bucket(BUCKET)
for obj in bucket.objects.all():
    print(obj.key)

dataset/checksum_blob.txt
dataset/coco.zip
dataset/yolo.zip
runs/custom_rtdetr_1743827923.zip
runs/custom_rtdetr_1743827923_weights.pt
runs/custom_yolo.zip
runs/custom_yolo_1743017449.zip
runs/custom_yolo_1743017449_weights.pt
runs/custom_yolo_1743736377.zip
runs/custom_yolo_1743736377_weights.pt
runs/custom_yolo_1743755320.zip
runs/custom_yolo_1743755320_weights.pt
runs/custom_yolo_1743767238.zip
runs/custom_yolo_1743767238_weights.pt
runs/custom_yolo_1743784218.zip
runs/custom_yolo_1743784218_weights.pt
runs/custom_yolo_1743896708.zip
runs/custom_yolo_1743896708_weights.pt
runs/rtdetr.zip
runs/rtdetr_1743885565.zip
runs/rtdetr_1743885565_weights.pt
runs/yolo.zip
runs/yolo_1742977346.zip
runs/yolo_1742977346_weights.pt
runs/yolo_1743657713.zip
runs/yolo_1743657713_weights.pt
runs/yolo_1743719798.zip
runs/yolo_1743719798_weights.pt
runs/yolo_1743844761.zip
runs/yolo_1743844761_weights.pt
runs/yolov8.zip


In [ ]:
import psycopg2
from urllib.parse import urlparse

DATABASE_URL = userdata.get("D298_DB_URI")

def execute_query(query, data=None, fetch=False):
    try:
        result = urlparse(DATABASE_URL)
        db_user = result.username
        db_password = result.password
        db_name = result.path[1:]
        db_host = result.hostname
        db_port = result.port

        with psycopg2.connect(
            database=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port
        ) as connection:
            with connection.cursor() as cursor:
                cursor.execute(query, data)
                if fetch:
                    results = cursor.fetchall()
                    return results
                else:
                    connection.commit()
                    return None

    except Exception as e:
        print(f"Error: {e}")
        return None

In [ ]:
query = 'SELECT * FROM "ModelResults" where "isActive"=True;'
results = execute_query(query, fetch=True)
models = []
if results:
    for row in results:
        if "manual_entry" in row[9]:
            continue
        models.append([row[0],row[11]])
else:
    print("No results found or error occurred.")
models

[[20, 'runs/custom_yolo_1743784218_weights.pt'],
 [16, 'runs/yolo_1743719798_weights.pt'],
 [17, 'runs/custom_yolo_1743736377_weights.pt'],
 [24, 'runs/custom_rtdetr_1743827923_weights.pt'],
 [26, 'runs/rtdetr_1743885565_weights.pt'],
 [27, 'runs/custom_yolo_1743896708_weights.pt'],
 [18, 'runs/custom_yolo_1743755320_weights.pt']]

In [ ]:
from ultralytics import YOLO, RTDETR

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
!rm -rf custom* model* yolo*

In [ ]:
models

[[20, 'runs/custom_yolo_1743784218_weights.pt'],
 [16, 'runs/yolo_1743719798_weights.pt'],
 [17, 'runs/custom_yolo_1743736377_weights.pt'],
 [24, 'runs/custom_rtdetr_1743827923_weights.pt'],
 [26, 'runs/rtdetr_1743885565_weights.pt'],
 [27, 'runs/custom_yolo_1743896708_weights.pt'],
 [18, 'runs/custom_yolo_1743755320_weights.pt']]

In [ ]:
# not_done = [25, 26]
not_done = None
# skip = None
skip = [24, 26]

In [ ]:
import shutil  # For moving directories

def upload_directory_to_s3(local_directory, bucket, s3_prefix):
    s3_uploader = boto3.client(
        "s3",
        region_name='us-east-1',
        aws_access_key_id=userdata.get("AWS_ACCESS_KEY_ID"),
        aws_secret_access_key=userdata.get("AWS_SECRET_ACCESS_KEY"),
    )
    for root, _, files in os.walk(local_directory):
        for filename in files:
            local_path = os.path.join(root, filename)
            relative_path = os.path.relpath(local_path, local_directory)
            s3_path = os.path.join(s3_prefix, relative_path)
            try:
                s3_uploader.upload_file(local_path, bucket, s3_path)
                print(f"--------------------------------------------Uploaded {local_path} to s3://{bucket}/{s3_path}")
            except Exception as e:
                print(f"--------------------------------------------Error uploading {local_path}: {e}")
                raise #Re-raise the exception to stop the loop


for m in models:
    print()
    print()
    print()
    print()
    print("-----------------------------------------------------------------------------------------------------------")
    print("-----------------------------------------------------------------------------------------------------------")
    print("-----------------------------------------------------------------------------------------------------------")
    print("-----------------------------------------------------------------------------------------------------------")
    print("-----------------------------------------------------------------------------------------------------------")
    # --- PREPARATION ---
    model_name = m[1].split("/")[1].split(".")[0]
    tfjs_folder_name = f"{model_name}_tfjs"
    model_file_name = "model.pt"

    !rm -rf model* yolo* custom* rtdetr*

    key = m[1]
    id = m[0]
    if not_done is not None:
        if id not in not_done:
            print(f"--------------------------------------------Skipping {key} as it is already done")
            continue
    if skip is not None:
        if id in skip:
            print(f"--------------------------------------------Skipping {key} as it is marked to be skipped")
            continue


    # --- DOWNLOAD ---
    try:
        s3.Bucket(BUCKET).download_file(key, model_file_name)
        print(f"--------------------------------------------Downloaded {key} from S3 to {model_file_name}")
    except Exception as e:
        print(f"--------------------------------------------Error downloading {key}: {e}")
        continue  # Skip to the next model

    # --- MODEL LOADING ---
    try:
        if "rtdetr" in key:
            model = RTDETR(model_file_name)
        else:
            model = YOLO(model_file_name)
    except Exception as e:
        print(f"--------------------------------------------Error loading model {key}: {e}")
        continue

    # --- EXPORT ---
    try:
        model.export(format="tfjs", half=True)
    except Exception as e:
        print(f"--------------------------------------------Error exporting model {key} to TFJS: {e}")
        continue

    # --- RENAMING ---
    try:
        shutil.move("model_web_model", tfjs_folder_name)
        print(f"--------------------------------------------Renamed model_web_model to {tfjs_folder_name}")
    except FileNotFoundError:
        print("--------------------------------------------Error: model_web_model directory not found after export.")
        continue
    except Exception as e:
        print(f"--------------------------------------------Error renaming directory: {e}")
        continue

    # --- UPLOAD ---
    s3_prefix = f"tfjs_models/{tfjs_folder_name}" #S3 path for tfjs model
    try:
        upload_directory_to_s3(tfjs_folder_name, BUCKET, s3_prefix)
        print(f"--------------------------------------------Uploaded {tfjs_folder_name} to s3://{BUCKET}/{s3_prefix}")
        model_s3_key = s3_prefix
    except Exception as e:
        print(f"--------------------------------------------Error uploading {tfjs_folder_name} to S3: {e}")
        continue

    # --- DATABASE UPDATE ---
    try:
        query = 'UPDATE "ModelResults" SET "tfjsS3Key" = %s WHERE id = %s;'
        data = (model_s3_key, id)
        print(data)
        execute_query(query, data=data)
        print(f"--------------------------------------------Updated database for id {id} with S3 key {model_s3_key}")
    except Exception as e:
        print(f"--------------------------------------------Error updating database for id {id}: {e}")






-----------------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------------
--------------------------------------------Downloaded runs/custom_yolo_1743784218_weights.pt from S3 to model.pt
Ultralytics 8.3.86 🚀 Python-3.11.11 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
YOLOv8s-custom summary (fused): 133 layers, 30,904,698 parameters, 0 gradients, 82.4 GFLOPs

PyTorch: starting from 'model.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 15, 8400) (59.4 MB)
requirements: Ultralytics requirements ['sng4onnx>=1.

100%|██████████| 1.11M/1.11M [00:00<00:00, 47.0MB/s]
Unzipping calibration_image_sample_data_20x128x128x3_float32.npy.zip to /content/calibration_image_sample_data_20x128x128x3_float32.npy...: 100%|██████████| 1/1 [00:00<00:00, 24.18file/s]



ONNX: starting export with onnx 1.16.1 opset 19...
ONNX: slimming with onnxslim 0.1.49...
ONNX: export success ✅ 9.5s, saved as 'model.onnx' (118.1 MB)
TensorFlow SavedModel: starting TFLite export with onnx2tf 1.26.3...
TensorFlow SavedModel: export success ✅ 279.2s, saved as 'model_saved_model' (295.8 MB)

TensorFlow GraphDef: starting export with tensorflow 2.19.0...
TensorFlow GraphDef: export success ✅ 4.5s, saved as 'model.pb' (118.1 MB)



TensorFlow.js: starting export with tensorflowjs 4.22.0...

TensorFlow.js: output node names: Identity:0
TensorFlow.js: running 'tensorflowjs_converter --input_format=tf_frozen_model --quantize_float16 --output_node_names=Identity:0 "model.pb" "model_web_model"'
TensorFlow.js: export success ✅ 6.8s, saved as 'model_web_model' (0.0 MB)

Export complete (295.1s)
Results saved to /content
Predict:         yolo predict task=detect model=model_web_model imgsz=640 half 
Validate:        yolo val task=detect model=model_web_model imgsz=640 data=/home/ubuntu/trainer/data/data.yaml half 
Visualize:       https://netron.app
--------------------------------------------Renamed model_web_model to custom_yolo_1743784218_weights_tfjs
--------------------------------------------Uploaded custom_yolo_1743784218_weights_tfjs/metadata.yaml to s3://sjsu-fall24-data298-team6-bucket/tfjs_models/custom_yolo_1743784218_weights_tfjs/metadata.yaml
--------------------------------------------Uploaded custom_yolo